# Explainer Notebook
This notebook was written by:
* Martin Petersson, s103619
* Christoffer Jensen, s103148

## Motivation
We have chosen to collect data from the Game of Thrones universe. The data was collected from various sites and API's, such as:
* *API's*:
    * https://api.got.show/api/
* *Sites*:
    * http://gameofthrones.wikia.com/wiki/

*Description of dataset: * 

> We have collected all the characters from Game of Thrones. For each character we have also collected their wikia page and which house they belong to (for community detection)

2) **Basic stats**. 
* Write about your choices in data cleaning and preprocessing
* Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)

3) **Tools, theory and analysis**. Describe the process of theory to insight 
* Talk about how you've worked with text, including regular expressions, unicode, etc.
* Describe which network science tools and data analysis strategies you've used, how those network science measures work, and why the tools you've chosen are right for the problem you're solving.
* How did you use the tools to understand your dataset?    

4) **Discussion.** Think critically about your creation
* What went well?,
* What is still missing? What could be improved?, Why? 

# References
* Barabási, Albert-László. Network Science. 1st ed. 2016. 
